In [ ]:
# start by importing libraries
#uncomment if error
#!pip install empath
import empath
import pandas as pd
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
import spacy
import seaborn as sns


In [ ]:
path = r"https://github.com/k3nnethfrancis/project-lion/raw/master/data/ken_stream.xlsx"
ken_stream = pd.read_excel(path)
ken_stream.head()


,content
0,Gm
1,Yea my thoughts exactly. I’ve interacted with ...
2,Yes please!
3,And yes for us I agree arweave is the better s...
4,Gm


In [ ]:
#get the number of messages by looking at the number of rows
print(len(ken_stream.index))

806


In [ ]:
#ensure it is all strings
ken_stream.content=ken_stream.content.astype(str)
# oes = ['content']
# for i in oes: # ensure all the text columns are strings if multiple cols
#     ken_stream[i] = ken_stream[i].astype(str)

In [ ]:
#make sure they are all sentences longer than 5 chars
ken_stream = ken_stream[ken_stream.content.str.len()>5] # strings longer than 5 characters only
ken_stream.head()

,content
1,Yea my thoughts exactly. I’ve interacted with ...
2,Yes please!
3,And yes for us I agree arweave is the better s...
5,https://ath.mirror.xyz/w2cxg5OP1OEcqvSgsEjSSyK...
6,Cc: <@539221068146278425>


In [ ]:
#calc the number of messages again
message_count = len(ken_stream.index)
print(message_count)

692


## Empath

The first cell is a basic example of empath given a message in the data 'content'.

In [ ]:
from empath import Empath
lexicon = Empath()

#get random variable
r = random.randrange(message_count)

#sentence to analyze
content = ken_stream['content'].iloc[r]

#analyze
nlp = lexicon.analyze(content, normalize=True)

#print the content
print(content)
#print topics with scores greater than 0.1
for key, v in nlp.items():
    if v > 0.01:
        print(key, v)

wow so just reading the abstract, are you essentially exploring how compounds like benzos alter the brains neural network?? Very cool!
surprise 0.047619047619047616
reading 0.047619047619047616
writing 0.047619047619047616


Ok. Now we will write several functions to ultimately loop through each message in `ken_stream.content` and append the results for each possible empath label to the dataframe.

In [ ]:
from pandas.io.json import json_normalize
import random

#loop through content and append the results to the dataframe

def get_emp(content):
    nlp = lexicon.analyze(content, normalize=True)
    return nlp

content = ken_stream.content.iloc[r]
emp_out = get_emp(content)

# print(content)

# print(emp_out)

def get_emp_cols(content):
    empath = get_emp(content)
    keys = list(empath.keys()) # get a list of the keys
    return keys

# emp_cols = get_emp_cols(content)

# print(emp_cols)

#add empath columns to df
def add_emp_cols(stream):
    emp_cols = get_emp_cols(stream.content.iloc[1])
    for i in emp_cols:
       stream[i] = stream[i]=np.nan
    return stream

ken_stream_update = add_emp_cols(ken_stream)

def apply_empath(stream):
    for row in range(len(stream.content)):
        content = stream['content'].iloc[row]
        emp = get_emp(content)
        new_df = pd.DataFrame(emp, index=[0])
        new_df.insert(0, 'content', content)
        ind = stream.iloc[[row]]
        new_df.index = ind.index
        stream.update(new_df)
    return stream

ken_empath = apply_empath(ken_stream_update)
ken_empath.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


,content,help,office,dance,money,wedding,domestic_work,sleep,medical_emergency,cold,...,weapon,children,monster,ocean,giving,contentment,writing,rural,positive_emotion,musical
1,Yea my thoughts exactly. I’ve interacted with ...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.018519,0.0
2,Yes please!,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,And yes for us I agree arweave is the better s...,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.058824,0.0
5,https://ath.mirror.xyz/w2cxg5OP1OEcqvSgsEjSSyK...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
6,Cc: <@539221068146278425>,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0


Ok. Now that we have a dataframe of each message with the respective empath label values, we'll want to define a few more functions to calculate some metrics about this data. Specifically, in this case, about my messages.

In [ ]:
# func 1: get the mean score of any given empath label

def get_empLabel_agg(empath_df, label='', func=['mean', 'min', 'max', 'median', 'sum', 'std']):
    if func == 'mean':
        return empath_df[label].mean()
    if func == 'min':
        return empath_df[label].min()
    if func == 'max':
        return empath_df[label].max()
    if func == 'median':
        return empath_df[label].median()
    if func == 'sum':
        return empath_df[label].sum()
    if func == 'std':
        return empath_df[label].std()
    elif func != ['mean', 'min', 'max', 'median', 'sum', 'std']:
        print('Please enter a function string: mean, min, max, median, sum, std.')
    

ken_help_agg = get_empLabel_agg(ken_empath, 'office', func='max')
print(ken_help_agg)


0.25


In [ ]:
#get aggregate empath voice for ken

def get_voice(stream):
    voice = {}
    for col in range(len(stream.columns)):
        if stream.columns[col]!='content':
            voice.update({stream.columns[col]:stream.iloc[:,1].mean()})
        else:
            continue
    return voice

kens_voice = get_voice(ken_empath)
kens_voice

{'achievement': 0.002313157196550621,
 'affection': 0.002313157196550621,
 'aggression': 0.002313157196550621,
 'air_travel': 0.002313157196550621,
 'alcohol': 0.002313157196550621,
 'ancient': 0.002313157196550621,
 'anger': 0.002313157196550621,
 'animal': 0.002313157196550621,
 'anonymity': 0.002313157196550621,
 'anticipation': 0.002313157196550621,
 'appearance': 0.002313157196550621,
 'art': 0.002313157196550621,
 'attractive': 0.002313157196550621,
 'banking': 0.002313157196550621,
 'beach': 0.002313157196550621,
 'beauty': 0.002313157196550621,
 'blue_collar_job': 0.002313157196550621,
 'body': 0.002313157196550621,
 'breaking': 0.002313157196550621,
 'business': 0.002313157196550621,
 'car': 0.002313157196550621,
 'celebration': 0.002313157196550621,
 'cheerfulness': 0.002313157196550621,
 'childish': 0.002313157196550621,
 'children': 0.002313157196550621,
 'cleaning': 0.002313157196550621,
 'clothing': 0.002313157196550621,
 'cold': 0.002313157196550621,
 'college': 0.002313

In [ ]:
#compare a given message to the overall voice
def voice_deviation(voice, message):
    emp = get_emp(message)
    delta = {key: voice[key] - emp.get(key, 0) for key in voice}
    return delta

message = ken_stream['content'].iloc[r]

message_dev = voice_deviation(kens_voice, message=message)

message_dev

{'achievement': 0.002313157196550621,
 'affection': 0.002313157196550621,
 'aggression': 0.002313157196550621,
 'air_travel': 0.002313157196550621,
 'alcohol': 0.002313157196550621,
 'ancient': 0.002313157196550621,
 'anger': 0.002313157196550621,
 'animal': 0.002313157196550621,
 'anonymity': 0.002313157196550621,
 'anticipation': 0.002313157196550621,
 'appearance': 0.002313157196550621,
 'art': 0.002313157196550621,
 'attractive': 0.002313157196550621,
 'banking': 0.002313157196550621,
 'beach': 0.002313157196550621,
 'beauty': 0.002313157196550621,
 'blue_collar_job': 0.002313157196550621,
 'body': 0.002313157196550621,
 'breaking': 0.002313157196550621,
 'business': 0.002313157196550621,
 'car': 0.002313157196550621,
 'celebration': 0.002313157196550621,
 'cheerfulness': 0.002313157196550621,
 'childish': 0.002313157196550621,
 'children': 0.002313157196550621,
 'cleaning': 0.002313157196550621,
 'clothing': 0.002313157196550621,
 'cold': 0.002313157196550621,
 'college': 0.002313